# Importação de bibliotecas

In [ ]:
# !pip install pandas --upgrade
# !pip install numpy --upgrade

import numpy as np
import pandas as pd
from IPython.display import display

## Importação do dataset (walmart-recruiting-store-sales-forecasting)

In [ ]:
import pandas as pd

# Definir o caminho base para os datasets raw no GitHub
github_base_url = 'https://raw.githubusercontent.com/gustavolima007/demand-predictor-walmart-MLOps/main/data/walmart-recruiting-store-sales-forecasting/'

# Lista para armazenar os dataframes
dfs = {}

# Nomes dos arquivos CSV a serem importados
csv_files = [
    'features.csv',
    'sampleSubmission.csv',
    'stores.csv',
    'test.csv',
    'train.csv'
]

# Importar cada arquivo CSV do GitHub
for file_name in csv_files:
    file_url = github_base_url + file_name
    df_name = file_name.replace('.csv', '') # Nome do dataframe será o nome do arquivo sem a extensão
    try:
        dfs[df_name] = pd.read_csv(file_url)
        print(f"Arquivo {file_name} importado como dataframe '{df_name}' do GitHub.")
    except Exception as e:
        print(f"Erro ao importar o arquivo {file_name} do GitHub: {e}")

# Agora, acesse e exiba as 5 primeiras linhas de cada dataframe
print("\nExibindo as 5 primeiras linhas de cada dataframe:")

# Acesse o dataframe 'stores' e exiba as 5 primeiras linhas
df_stores = dfs.get('stores')
if df_stores is not None:
    print("\nExibindo dataframe 'df_stores':")
    display(df_stores.head())

# Acesse o dataframe 'train' e exiba as 5 primeiras linhas
df_train = dfs.get('train')
if df_train is not None:
    print("\nExibindo dataframe 'df_train':")
    display(df_train.head())

# Acesse o dataframe 'test' e exiba as 5 primeiras linhas
df_test = dfs.get('test')
if df_test is not None:
    print("\nExibindo dataframe 'df_test':")
    display(df_test.head())

# Acesse o dataframe 'sampleSubmission' e exiba as 5 primeiras linhas
df_sampleSubmission = dfs.get('sampleSubmission')
if df_sampleSubmission is not None:
    print("\nExibindo dataframe 'df_sampleSubmission':")
    display(df_sampleSubmission.head())

# Acesse o dataframe 'features' e exiba as 5 primeiras linhas
df_features = dfs.get('features')
if df_features is not None:
    print("\nExibindo dataframe 'df_features':")
    display(df_features.head())

## Unificação dos Dados (Merge)

In [ ]:
import pandas as pd

# 1. Mesclar df_train com df_stores com base na coluna 'Store'
df_train_stores = pd.merge(df_train, df_stores, on='Store', how='left')

# 2. Mesclar o resultado com df_features com base em 'Store' e 'Date'
df_train_merged = pd.merge(df_train_stores, df_features, on=['Store', 'Date'], how='left')

# 3. Verificar o resultado
print("\nShape de df_train_merged:", df_train_merged.shape)
print("\nExibindo as 5 primeiras linhas do dataframe df_train_merged:")
display(df_train_merged.head())

# 4. Verificar se há valores ausentes
print("\nValores ausentes por coluna:")
print(df_train_merged.isnull().sum())

In [ ]:
# Para o conjunto de teste
df_test_merged = pd.merge(df_test, df_stores, on='Store', how='left')
df_test_merged = pd.merge(df_test_merged, df_features, on=['Store', 'Date', 'IsHoliday'], how='left')

print("\nExibindo as 5 primeiras linhas do dataframe df_test_merged:")
display(df_test_merged.head())

# Limpeza de Dados

Visualização dos dataframe

In [ ]:
print("Informações sobre df_train_merged:")
df_train_merged.info()

print("\nInformações sobre df_test_merged:")
df_test_merged.info()

In [ ]:
# Converter 'Date' para datetime
df_train_merged['Date'] = pd.to_datetime(df_train_merged['Date'])
df_test_merged['Date'] = pd.to_datetime(df_test_merged['Date'])

df_train_merged.info()
df_test_merged.info()

In [ ]:
print("--- Estatísticas Descritivas: df_train_merged ---")
display(df_train_merged.describe(include='all'))

print("\n--- Estatísticas Descritivas: df_test_merged ---")
display(df_test_merged.describe(include='all'))

## Tratamento de Weekly_Sales Negativas e Colunas MarkDown

In [ ]:
num_vendas_negativas = (df_train_merged['Weekly_Sales'] < 0).sum()
print(f"Número de Weekly_Sales negativas: {num_vendas_negativas}")
if num_vendas_negativas > 0:
    print("Substituindo Weekly_Sales negativas por 0...")
    df_train_merged.loc[df_train_merged['Weekly_Sales'] < 0, 'Weekly_Sales'] = 0
    print(f"Verificação - Novo mínimo de Weekly_Sales: {df_train_merged['Weekly_Sales'].min()}")
else:
    print("Nenhuma Weekly_Sales negativa encontrada.")

In [ ]:
# 1. Tratamento de Weekly_Sales negativas em df_train_merged
print("--- Tratando Weekly_Sales negativas em df_train_merged ---")
num_vendas_negativas = (df_train_merged['Weekly_Sales'] < 0).sum()
if num_vendas_negativas > 0:
    print(f"Encontradas {num_vendas_negativas} Weekly_Sales negativas. Substituindo por 0...")
    df_train_merged.loc[df_train_merged['Weekly_Sales'] < 0, 'Weekly_Sales'] = 0
    print(f"Verificação - Novo mínimo de Weekly_Sales: {df_train_merged['Weekly_Sales'].min()}")
else:
    print("Nenhuma Weekly_Sales negativa encontrada em df_train_merged.")

# 2. Tratamento de MarkDowns negativos

markdown_cols_to_check = ['MarkDown2', 'MarkDown3']

# Tratamento para df_train_merged
print("\n--- Tratando MarkDowns negativos em df_train_merged ---")
for col in markdown_cols_to_check:
    if col in df_train_merged.columns:
        num_neg_markdown_train = (df_train_merged[col] < 0).sum()
        if num_neg_markdown_train > 0:
            print(f"Encontrados {num_neg_markdown_train} valores negativos em {col}. Substituindo por 0...")
            df_train_merged.loc[df_train_merged[col] < 0, col] = 0
            print(f"Verificação - Novo mínimo de {col}: {df_train_merged[col].min()}")
        else:
            print(f"Nenhum valor negativo encontrado em {col} no df_train_merged.")
    else:
        print(f"Coluna {col} não encontrada em df_train_merged.")

# Tratamento para df_test_merged
print("\n--- Tratando MarkDowns negativos em df_test_merged ---")
for col in markdown_cols_to_check:
    if col in df_test_merged.columns:
        num_neg_markdown_test = (df_test_merged[col] < 0).sum()
        if num_neg_markdown_test > 0:
            print(f"Encontrados {num_neg_markdown_test} valores negativos em {col}. Substituindo por 0...")
            df_test_merged.loc[df_test_merged[col] < 0, col] = 0
            print(f"Verificação - Novo mínimo de {col}: {df_test_merged[col].min()}")
        else:
            print(f"Nenhum valor negativo encontrado em {col} no df_test_merged.")
    else:
        print(f"Coluna {col} não encontrada em df_test_merged.")

print("\nTratamento de valores negativos concluído!")

In [ ]:
 # Validando valores negaticos:

for i in range(1, 6):
    col_name = f'MarkDown{i}'
    print(f"\nAnálise de {col_name} em df_train_merged:")
    print(f"Número de valores negativos: {(df_train_merged[col_name] < 0).sum()}")
    print(df_train_merged[df_train_merged[col_name] < 0][col_name].describe())

## Verificação de Duplicatas

In [ ]:
# Para df_train_merged
num_duplicatas_treino = df_train_merged.duplicated().sum()
print(f"\nNúmero de linhas duplicadas em df_train_merged: {num_duplicatas_treino}")
if num_duplicatas_treino > 0:
    print(f"Removendo {num_duplicatas_treino} linhas duplicadas de df_train_merged...")
    df_train_merged.drop_duplicates(inplace=True)
    df_train_merged.reset_index(drop=True, inplace=True) # Opcional: resetar o índice
    print(f"Novo shape de df_train_merged: {df_train_merged.shape}")

# Para df_test_merged
num_duplicatas_teste = df_test_merged.duplicated().sum()
print(f"\nNúmero de linhas duplicadas em df_test_merged: {num_duplicatas_teste}")
if num_duplicatas_teste > 0:
    print(f"Removendo {num_duplicatas_teste} linhas duplicadas de df_test_merged...")
    df_test_merged.drop_duplicates(inplace=True)
    df_test_merged.reset_index(drop=True, inplace=True) # Opcional: resetar o índice
    print(f"Novo shape de df_test_merged: {df_test_merged.shape}")

## Tratamento de Valores Ausentes (NaNs)

Verificando NaNs

In [ ]:
print("--- Verificando NaNs em df_train_merged ANTES do tratamento ---")
print(df_train_merged.isnull().sum())

print("\n--- Verificando NaNs em df_test_merged ANTES do tratamento ---")
print(df_test_merged.isnull().sum())

Tratamento de NaNs nas colunas MarkDown1 a MarkDown5

In [ ]:
# 1. Tratamento de NaNs nas colunas MarkDown1 a MarkDown5
# Estratégia: Preencher NaNs com 0, pois indicam ausência de promoção.
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']

print("\n--- Tratando NaNs nas colunas MarkDown (preenchendo com 0) ---")
for col in markdown_cols:
    if col in df_train_merged.columns:
        df_train_merged[col] = df_train_merged[col].fillna(0)
    if col in df_test_merged.columns:
        df_test_merged[col] = df_test_merged[col].fillna(0)
print("Preenchimento de NaNs nas colunas MarkDown concluído.")

# 2. Tratamento de NaNs nas colunas CPI e Unemployment
# Estratégia: Preenchimento progressivo (ffill) e depois regressivo (bfill), agrupado por 'Store'.
cols_to_ffill_bfill = ['CPI', 'Unemployment']

print("\n--- Tratando NaNs em CPI e Unemployment (ffill e bfill por Loja) ---")

# Para df_train_merged
print("Tratando df_train_merged para CPI e Unemployment...")
for col in cols_to_ffill_bfill:
    if col in df_train_merged.columns:
        # Agrupa por 'Store', aplica ffill e depois bfill dentro de cada grupo
        df_train_merged[col] = df_train_merged.groupby('Store')[col].ffill()
        df_train_merged[col] = df_train_merged.groupby('Store')[col].bfill()
    else:
        print(f"Coluna {col} não encontrada em df_train_merged.")

# Para df_test_merged
print("Tratando df_test_merged para CPI e Unemployment...")
for col in cols_to_ffill_bfill:
    if col in df_test_merged.columns:
        # Agrupa por 'Store', aplica ffill e depois bfill dentro de cada grupo
        df_test_merged[col] = df_test_merged.groupby('Store')[col].ffill()
        df_test_merged[col] = df_test_merged.groupby('Store')[col].bfill()
    else:
        print(f"Coluna {col} não encontrada em df_test_merged.")
print("Preenchimento de NaNs em CPI e Unemployment concluído.")

# 3. Verificação final de NaNs
print("\n--- Verificando NaNs em df_train_merged APÓS o tratamento ---")
print(df_train_merged.isnull().sum())

print("\n--- Verificando NaNs em df_test_merged APÓS o tratamento ---")
print(df_test_merged.isnull().sum())

print("\nTratamento de valores ausentes (NaNs) concluído!")

## Transformação de Variáveis Categóricas (One-Hot Encoding)

### Alteração de colunas

In [ ]:
print("--- Transformando a coluna 'Type' (One-Hot Encoding) ---")

# Aplicar em df_train_merged
df_train_merged = pd.get_dummies(df_train_merged, columns=['Type'], prefix='Type')
print("Coluna 'Type' transformada em df_train_merged.")
display(df_train_merged.head(2)) # Mostrar algumas linhas para ver as novas colunas

# Aplicar em df_test_merged
df_test_merged = pd.get_dummies(df_test_merged, columns=['Type'], prefix='Type')
print("\nColuna 'Type' transformada em df_test_merged.")
display(df_test_merged.head(2)) # Mostrar algumas linhas para ver as novas colunas

Transformação da Variável Booleana

In [ ]:
import pandas as pd

# 1. Mesclar df_train com df_stores com base na coluna 'Store'
df_train_stores = pd.merge(df_train, df_stores, on='Store', how='left')

# 2. Mesclar o resultado com df_features com base em 'Store' e 'Date'
df_train_merged = pd.merge(df_train_stores, df_features, on=['Store', 'Date'], how='left')

# 3. Verificar o resultado
print("\nShape de df_train_merged:", df_train_merged.shape)
print("\nExibindo as 5 primeiras linhas do dataframe df_train_merged:")
display(df_train_merged.head())

# 4. Verificar se há valores ausentes
print("\nValores ausentes por coluna:")
print(df_train_merged.isnull().sum())

# 5. Tratar a coluna 'IsHoliday'
print("\n--- Tratando a coluna 'IsHoliday' ---")

# Verificar se 'IsHoliday_x' e 'IsHoliday_y' são consistentes (opcional, para debugging)
if 'IsHoliday_x' in df_train_merged.columns and 'IsHoliday_y' in df_train_merged.columns:
    discrepancias = df_train_merged[df_train_merged['IsHoliday_x'] != df_train_merged['IsHoliday_y']]
    if not discrepancias.empty:
        print(f"ATENÇÃO: {len(discrepancias)} linhas com discrepâncias entre 'IsHoliday_x' e 'IsHoliday_y'.")
        print("Exemplo de discrepâncias:")
        display(discrepancias[['Date', 'IsHoliday_x', 'IsHoliday_y']].head())
    # Usar 'IsHoliday_x' como a fonte principal
    df_train_merged['IsHoliday'] = df_train_merged['IsHoliday_x']
else:
    print("Uma ou ambas as colunas 'IsHoliday_x' ou 'IsHoliday_y' não estão presentes.")

# 6. Transformar 'IsHoliday' para int (0/1)
df_train_merged['IsHoliday'] = df_train_merged['IsHoliday'].astype(int)
print("Coluna 'IsHoliday' transformada para int em df_train_merged.")

# 7. Repetir para df_test_merged
df_test_stores = pd.merge(df_test, df_stores, on='Store', how='left')
df_test_merged = pd.merge(df_test_stores, df_features, on=['Store', 'Date'], how='left')
df_test_merged['IsHoliday'] = df_test_merged['IsHoliday_x'].astype(int)
print("Coluna 'IsHoliday' transformada para int em df_test_merged.")

# 8. Exibir amostra para verificação
print("\nExibindo as 3 primeiras linhas de 'Date' e 'IsHoliday' em df_train_merged:")
display(df_train_merged[['Date', 'IsHoliday']].head(3))

# Engenharia de Features



## Criação de Features Temporais

In [ ]:
import pandas as pd

# 1. Verificar e converter 'Date' para datetime em df_train_merged
print("--- Verificando e convertendo 'Date' em df_train_merged ---")
if 'Date' not in df_train_merged.columns:
    print("ERRO: Coluna 'Date' não encontrada em df_train_merged.")
    print("Colunas disponíveis:", df_train_merged.columns.tolist())
else:
    if not pd.api.types.is_datetime64_any_dtype(df_train_merged['Date']):
        df_train_merged['Date'] = pd.to_datetime(df_train_merged['Date'], errors='coerce')
        print("Coluna 'Date' convertida para datetime em df_train_merged.")
    else:
        print("Coluna 'Date' já está em formato datetime em df_train_merged.")

# 2. Verificar e converter 'Date' para datetime em df_test_merged
print("\n--- Verificando e convertendo 'Date' em df_test_merged ---")
if 'Date' not in df_test_merged.columns:
    print("ERRO: Coluna 'Date' não encontrada em df_test_merged.")
    print("Colunas disponíveis:", df_test_merged.columns.tolist())
else:
    if not pd.api.types.is_datetime64_any_dtype(df_test_merged['Date']):
        df_test_merged['Date'] = pd.to_datetime(df_test_merged['Date'], errors='coerce')
        print("Coluna 'Date' convertida para datetime em df_test_merged.")
    else:
        print("Coluna 'Date' já está em formato datetime em df_test_merged.")

# 3. Função para criar features temporais
print("\n--- Atualizando a função para criar mais Features Temporais ---")

def criar_features_temporais_v2(df):
    df_copy = df.copy()
    
    # Features básicas
    df_copy['Year'] = df_copy['Date'].dt.year
    df_copy['Month'] = df_copy['Date'].dt.month
    df_copy['Day'] = df_copy['Date'].dt.day
    df_copy['WeekOfYear'] = df_copy['Date'].dt.isocalendar().week.astype(int)
    df_copy['DayOfWeek'] = df_copy['Date'].dt.dayofweek  # Segunda=0, Domingo=6
    df_copy['DayOfYear'] = df_copy['Date'].dt.dayofyear
    
    # Novas features
    df_copy['Quarter'] = df_copy['Date'].dt.quarter
    df_copy['IsMonthStart'] = df_copy['Date'].dt.is_month_start.astype(int)
    df_copy['IsMonthEnd'] = df_copy['Date'].dt.is_month_end.astype(int)
    df_copy['IsYearStart'] = df_copy['Date'].dt.is_year_start.astype(int)
    df_copy['IsYearEnd'] = df_copy['Date'].dt.is_year_end.astype(int)
    
    # Semana do Mês
    df_copy['WeekOfMonth'] = (df_copy['Date'].dt.day - 1) // 7 + 1

    return df_copy

# 4. Aplicar a função em df_train_merged
if 'Date' in df_train_merged.columns and pd.api.types.is_datetime64_any_dtype(df_train_merged['Date']):
    df_train_merged = criar_features_temporais_v2(df_train_merged)
    print("\nNovas features temporais adicionadas a df_train_merged.")
    cols_to_display = ['Date', 'Year', 'Month', 'Day', 'WeekOfYear', 'DayOfWeek', 'DayOfYear', 
                       'Quarter', 'IsMonthStart', 'IsMonthEnd', 'IsYearStart', 'IsYearEnd', 'WeekOfMonth']
    cols_to_display = [col for col in cols_to_display if col in df_train_merged.columns]
    display(df_train_merged[cols_to_display].head())
else:
    print("ERRO: Coluna 'Date' não encontrada ou não é datetime em df_train_merged. Novas features temporais não criadas.")

# 5. Aplicar a função em df_test_merged
if 'Date' in df_test_merged.columns and pd.api.types.is_datetime64_any_dtype(df_test_merged['Date']):
    df_test_merged = criar_features_temporais_v2(df_test_merged)
    print("\nNovas features temporais adicionadas a df_test_merged.")
    cols_to_display_test = [col for col in cols_to_display if col in df_test_merged.columns]
    display(df_test_merged[cols_to_display_test].head())
else:
    print("ERRO: Coluna 'Date' não encontrada ou não é datetime em df_test_merged. Novas features temporais não criadas.")

## Criação de Features de Lag (Defasagem)

In [ ]:
print("--- Criando Features de Lag (com ajuste para FutureWarning) ---")

# 1. Preparar dataframes temporários para a criação de lags
# Selecionar colunas chave e a variável alvo (Weekly_Sales apenas do treino)
df_train_temp_for_lag = df_train_merged[['Store', 'Dept', 'Date', 'Weekly_Sales']].copy()
df_test_temp_for_lag = df_test_merged[['Store', 'Dept', 'Date']].copy()

# Adicionar coluna Weekly_Sales com np.nan para o teste (para consistência de tipo float)
df_test_temp_for_lag['Weekly_Sales'] = np.nan #

# Adicionar um identificador para separar depois
df_train_temp_for_lag['is_train'] = 1
df_test_temp_for_lag['is_train'] = 0

# 2. Concatenar os dataframes temporários
# Este é o ponto onde o FutureWarning ocorria. O ajuste acima com np.nan deve ajudar.
df_combined = pd.concat([df_train_temp_for_lag, df_test_temp_for_lag], ignore_index=True)

# 3. Ordenar corretamente ANTES de aplicar o shift dentro dos grupos
df_combined.sort_values(by=['Store', 'Dept', 'Date'], inplace=True)

# 4. Definir os lags que queremos criar
lags_to_create = [1, 2, 3, 4, 12, 26, 52] # Ex: 1 semana, 2 sem, ..., 1 ano

for lag in lags_to_create:
    # O groupby garante que o shift é feito dentro de cada série individual de Loja/Departamento
    df_combined[f'Weekly_Sales_lag_{lag}'] = df_combined.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(lag)

print(f"Features de lag {lags_to_create} criadas no dataframe combinado.")

# 5. Separar de volta em treino e teste
df_train_with_lags = df_combined[df_combined['is_train'] == 1].copy()
df_test_with_lags = df_combined[df_combined['is_train'] == 0].copy()

# Remover colunas auxiliares
df_train_with_lags.drop(columns=['is_train'], inplace=True)
# Para o teste, remover também a coluna 'Weekly_Sales' que foi preenchida com np.nan
df_test_with_lags.drop(columns=['is_train', 'Weekly_Sales'], inplace=True)

# 6. Juntar as novas colunas de lag de volta aos dataframes originais
# Merge para df_train_merged (usando as colunas originais para garantir a junção correta)
df_train_merged = pd.merge(
    df_train_merged,
    df_train_with_lags, # Contém Store, Dept, Date, Weekly_Sales (original) e os lags
    on=['Store', 'Dept', 'Date', 'Weekly_Sales'], # Chaves para garantir correspondência exata
    how='left'
)

# Merge para df_test_merged
df_test_merged = pd.merge(
    df_test_merged,
    df_test_with_lags, # Contém Store, Dept, Date e os lags
    on=['Store', 'Dept', 'Date'], # Chaves para o teste
    how='left'
)

print("Features de lag adicionadas a df_train_merged e df_test_merged.")

# 7. Verificar as novas colunas e os NaNs introduzidos pelos lags (opcional, mas recomendado)
print("\nExemplo de features de lag em df_train_merged (Loja 1, Dept 1):")
cols_to_show_train = ['Date', 'Weekly_Sales'] + [col for col in df_train_merged.columns if 'Weekly_Sales_lag' in col]
# Filtrando para uma loja/departamento específico para facilitar a visualização dos lags
display(df_train_merged[(df_train_merged['Store']==1) & (df_train_merged['Dept']==1)][cols_to_show_train].head(10))

print("\nExemplo de features de lag em df_test_merged (Loja 1, Dept 1):")
cols_to_show_test = ['Date'] + [col for col in df_test_merged.columns if 'Weekly_Sales_lag' in col]
display(df_test_merged[(df_test_merged['Store']==1) & (df_test_merged['Dept']==1)][cols_to_show_test].head(10))

print("\nVerificando NaNs introduzidos pelos lags em df_train_merged:")
print(df_train_merged[[col for col in df_train_merged.columns if 'Weekly_Sales_lag' in col]].isnull().sum().sort_values(ascending=False))

print("\nVerificando NaNs introduzidos pelos lags em df_test_merged:")
print(df_test_merged[[col for col in df_test_merged.columns if 'Weekly_Sales_lag' in col]].isnull().sum().sort_values(ascending=False))

print("\nCriação de features de lag concluída!")

## Criação de Features de Janela Móvel (Rolling Window)

In [ ]:
# # Converter a coluna 'Date' para datetime, se ainda não estiver
# df_train['Date'] = pd.to_datetime(df_train['Date'])
# df_test['Date'] = pd.to_datetime(df_test['Date'])

# Definir a data de corte para a divisão treino/validação
train_end_date = '2012-07-31'
val_start_date = '2012-08-01'

# Dividir df_train em treino e validação com base na data
df_train_subset = df_train[df_train['Date'] <= train_end_date].copy()
df_val_subset = df_train[(df_train['Date'] >= val_start_date) & (df_train['Date'] <= '2012-10-26')].copy()

print(f"Treino: {df_train_subset['Date'].min()} a {df_train_subset['Date'].max()} ({len(df_train_subset)} linhas)")
print(f"Validação: {df_val_subset['Date'].min()} a {df_val_subset['Date'].max()} ({len(df_val_subset)} linhas)")
print(f"Teste: {df_test['Date'].min()} a {df_test['Date'].max()} ({len(df_test)} linhas)")

# --- Criando Features de Janela Móvel ---
window_sizes = [4, 8, 12, 26, 52]
stats_to_calculate = ['mean', 'median', 'sum', 'std']

# 1. Calcular features para o conjunto de treino
df_train_subset = df_train_subset.sort_values(['Store', 'Dept', 'Date'])
for window in window_sizes:
    for stat in stats_to_calculate:
        new_col_name = f'Weekly_Sales_roll_{stat}_{window}'
        df_train_subset[new_col_name] = df_train_subset.groupby(['Store', 'Dept'])['Weekly_Sales'].transform(
            lambda s: s.rolling(window=window, min_periods=1).agg(stat).shift(1)
        )
        print(f"Criada feature no treino: {new_col_name}")

# 2. Calcular features para o conjunto de validação
df_val_subset = df_val_subset.sort_values(['Store', 'Dept', 'Date'])
for window in window_sizes:
    for stat in stats_to_calculate:
        new_col_name = f'Weekly_Sales_roll_{stat}_{window}'
        # Concatenar dados de treino com validação para garantir continuidade temporal
        df_temp = pd.concat([
            df_train_subset[['Store', 'Dept', 'Date', 'Weekly_Sales']],
            df_val_subset[['Store', 'Dept', 'Date']].assign(Weekly_Sales=np.nan)
        ])
        df_temp = df_temp.sort_values(['Store', 'Dept', 'Date'])
        df_temp[new_col_name] = df_temp.groupby(['Store', 'Dept'])['Weekly_Sales'].transform(
            lambda s: s.rolling(window=window, min_periods=1).agg(stat).shift(1)
        )
        # Atribuir as features ao conjunto de validação
        df_val_subset[new_col_name] = df_temp[df_temp['Date'] >= val_start_date][new_col_name].values
        print(f"Criada feature na validação: {new_col_name}")

# 3. Calcular features para o conjunto de teste
df_test = df_test.sort_values(['Store', 'Dept', 'Date'])
for window in window_sizes:
    for stat in stats_to_calculate:
        new_col_name = f'Weekly_Sales_roll_{stat}_{window}'
        # Concatenar dados de treino com teste para garantir continuidade temporal
        df_temp = pd.concat([
            df_train[['Store', 'Dept', 'Date', 'Weekly_Sales']],
            df_test[['Store', 'Dept', 'Date']].assign(Weekly_Sales=np.nan)
        ])
        df_temp = df_temp.sort_values(['Store', 'Dept', 'Date'])
        df_temp[new_col_name] = df_temp.groupby(['Store', 'Dept'])['Weekly_Sales'].transform(
            lambda s: s.rolling(window=window, min_periods=1).agg(stat).shift(1)
        )
        # Atribuir as features ao conjunto de teste
        df_test[new_col_name] = df_temp[df_temp['Date'] >= '2012-11-02'][new_col_name].values
        print(f"Criada feature no teste: {new_col_name}")

# 4. Verificar e tratar NaNs
print("\nVerificando NaNs no df_train_subset:")
print(df_train_subset[[col for col in df_train_subset.columns if 'Weekly_Sales_roll_' in col]].isnull().sum())
print("\nVerificando NaNs no df_val_subset:")
print(df_val_subset[[col for col in df_val_subset.columns if 'Weekly_Sales_roll_' in col]].isnull().sum())
print("\nVerificando NaNs no df_test:")
print(df_test[[col for col in df_test.columns if 'Weekly_Sales_roll_' in col]].isnull().sum())

# Tratar NaNs (preencher com a média por grupo)
for col in [col for col in df_train_subset.columns if 'Weekly_Sales_roll_' in col]:
    df_train_subset[col] = df_train_subset.groupby(['Store', 'Dept'])[col].transform(lambda x: x.fillna(x.mean()))
    df_val_subset[col] = df_val_subset.groupby(['Store', 'Dept'])[col].transform(lambda x: x.fillna(x.mean()))
    df_test[col] = df_test.groupby(['Store', 'Dept'])[col].transform(lambda x: x.fillna(x.mean()))

print("\nCriação de features de janela móvel concluída!")

## Tratamento Final de NaNs nas Features Criadas

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# NaNs na validação e teste, para evitar data leakage.
print("\n--- Iniciando Tratamento de NaNs para Features de Lag e Janela Móvel ---")

# Identificar as colunas de janela móvel e lag em df_train_subset
# (Assumindo que df_val_subset e df_test terão as mesmas colunas de features)
if 'df_train_subset' in locals():
    cols_roll = [col for col in df_train_subset.columns if 'Weekly_Sales_roll_' in col]
    cols_lag = [col for col in df_train_subset.columns if 'Weekly_Sales_lag_' in col]
    cols_to_treat = cols_roll + cols_lag
else:
    print("ERRO: df_train_subset não está definido. Interrompendo tratamento de NaNs.")

    cols_to_treat = [] # Evitar erro mais abaixo

if cols_to_treat:
    # Passo 1: Calcular as médias por grupo apenas no conjunto de treino (df_train_subset)
    print("\nCalculando médias por grupo no conjunto de treino para preenchimento...")
    group_means = {}
    for col in cols_to_treat:
        if col in df_train_subset.columns:
            means = df_train_subset.groupby(['Store', 'Dept'])[col].mean()
            group_means[col] = means
        else:
            print(f"AVISO: Coluna {col} não encontrada em df_train_subset para cálculo de média.")

    # Passo 2: Preencher NaNs em df_train_subset usando as médias do próprio conjunto
    print("\nTratando NaNs em df_train_subset...")
    for col in cols_to_treat:
        if col in df_train_subset.columns:
            # Usar a média do grupo do próprio treino para preencher NaNs do treino
            df_train_subset[f'{col}_filled_temp'] = df_train_subset.groupby(['Store', 'Dept'])[col].transform(lambda x: x.fillna(x.mean()))
            df_train_subset[col] = df_train_subset[f'{col}_filled_temp'].fillna(0) # Fallback para 0 se a média do grupo for NaN
            df_train_subset.drop(columns=[f'{col}_filled_temp'], inplace=True)
    print("NaNs preenchidos em df_train_subset.")

    # Passo 3: Preencher NaNs em df_val_subset usando as médias calculadas do conjunto de treino
    print("\nTratando NaNs em df_val_subset...")
    if 'df_val_subset' in locals():
        for col in cols_to_treat:
            if col in df_val_subset.columns and col in group_means:
                # Mapear as médias do treino para o conjunto de validação
                mapped_means = df_val_subset.set_index(['Store', 'Dept']).index.map(group_means[col])
                df_val_subset[col] = df_val_subset[col].fillna(pd.Series(mapped_means, index=df_val_subset.index))
                df_val_subset[col] = df_val_subset[col].fillna(0) # Fallback para 0
            elif col not in group_means and col in df_val_subset.columns:
                 print(f"AVISO: Não há média de treino para {col}. Preenchendo NaNs em df_val_subset com 0.")
                 df_val_subset[col] = df_val_subset[col].fillna(0)

        print("NaNs preenchidos em df_val_subset.")
    else:
        print("AVISO: df_val_subset não definido. Pule o tratamento de NaNs para validação.")


    # Passo 4: Preencher NaNs em df_test usando as médias calculadas do conjunto de treino
    print("\nTratando NaNs em df_test...")
    if 'df_test' in locals():
        for col in cols_to_treat:
            if col in df_test.columns and col in group_means:
                # Mapear as médias do treino para o conjunto de teste
                mapped_means = df_test.set_index(['Store', 'Dept']).index.map(group_means[col])
                df_test[col] = df_test[col].fillna(pd.Series(mapped_means, index=df_test.index))
                df_test[col] = df_test[col].fillna(0) # Fallback para 0
            elif col not in group_means and col in df_test.columns:
                print(f"AVISO: Não há média de treino para {col}. Preenchendo NaNs em df_test com 0.")
                df_test[col] = df_test[col].fillna(0)
        print("NaNs preenchidos em df_test.")
    else:
        print("AVISO: df_test não definido. Pule o tratamento de NaNs para teste.")


    # Verificação final de NaNs
    print("\n--- Verificando NaNs APÓS o tratamento final ---")
    if 'df_train_subset' in locals():
        print("\nFeatures em df_train_subset:")
        print(df_train_subset[cols_to_treat].isnull().sum())
    if 'df_val_subset' in locals():
        print("\nFeatures em df_val_subset:")
        print(df_val_subset[cols_to_treat].isnull().sum())
    if 'df_test' in locals():
        print("\nFeatures em df_test:")
        print(df_test[cols_to_treat].isnull().sum())
    print("\nTratamento final de NaNs concluído!")
else:
    print("Nenhuma coluna de lag ou rolling para tratar NaNs foi identificada em df_train_subset.")

In [ ]:
# --- Tratando NaNs Finais nas Features de Janela Móvel e Lag (usando médias do treino para evitar leakage) ---
print("--- Tratando NaNs Finais nas Features de Janela Móvel e Lag (usando médias do treino para evitar leakage) ---")


# Identificar as colunas de janela móvel e lag
cols_roll = [col for col in df_train_subset.columns if 'Weekly_Sales_roll_' in col]
cols_lag = [col for col in df_train_subset.columns if 'Weekly_Sales_lag_' in col]
cols_to_treat = cols_roll + cols_lag # Lista combinada de colunas a serem tratadas

# Passo 1: Calcular as médias por grupo apenas no conjunto de treino (df_train_subset)
print("\nCalculando médias por grupo no conjunto de treino...")
group_means = {}
for col in cols_to_treat: # Usar a lista combinada
    if col in df_train_subset.columns:
        means = df_train_subset.groupby(['Store', 'Dept'])[col].mean()
        group_means[col] = means

# Passo 2: Preencher NaNs em df_train_subset usando as médias do próprio conjunto e 0 como fallback
print("\nTratando NaNs em df_train_subset...")
for col in cols_to_treat:
    if col in df_train_subset.columns:
        # Cria uma coluna temporária com as médias do grupo para preenchimento
        # Isso evita o warning de SettingWithCopyWarning em algumas versões do pandas com transform
        df_train_subset[f'{col}_group_mean_fill'] = df_train_subset.groupby(['Store', 'Dept'])[col].transform(lambda x: x.fillna(x.mean()))
        # Preenche a coluna original usando a coluna temporária
        df_train_subset[col] = df_train_subset[f'{col}_group_mean_fill']
        # Preenche quaisquer NaNs restantes (se a média do grupo também era NaN) com 0
        df_train_subset[col] = df_train_subset[col].fillna(0)
        # Remove a coluna temporária
        df_train_subset.drop(columns=[f'{col}_group_mean_fill'], inplace=True)
print("NaNs preenchidos em df_train_subset.")

# Passo 3: Preencher NaNs em df_val_subset usando as médias calculadas do conjunto de treino
print("\nTratando NaNs em df_val_subset...")
for col in cols_to_treat:
    if col in df_val_subset.columns and col in group_means:
        # Mapear as médias do treino (group_means) para o conjunto de validação
        # Criar uma série de médias alinhada com o índice de df_val_subset
        mapped_means_val = df_val_subset.set_index(['Store', 'Dept']).index.map(group_means[col])
        # Atribuir ao índice correto para evitar desalinhamento ao usar fillna com uma Series
        mapped_means_val_series = pd.Series(mapped_means_val, index=df_val_subset.index)
        
        df_val_subset[col] = df_val_subset[col].fillna(mapped_means_val_series)
        df_val_subset[col] = df_val_subset[col].fillna(0) # Fallback para 0
    elif col in df_val_subset.columns: # Se a coluna existe mas não há média do treino para ela
        print(f"AVISO: Não há média de treino para {col} (ou coluna não em group_means). Preenchendo NaNs em df_val_subset com 0.")
        df_val_subset[col] = df_val_subset[col].fillna(0)
print("NaNs preenchidos em df_val_subset.")

# Passo 4: Preencher NaNs em df_test usando as médias calculadas do conjunto de treino
print("\nTratando NaNs em df_test...")
for col in cols_to_treat:
    if col in df_test.columns and col in group_means:
        # Mapear as médias do treino (group_means) para o conjunto de teste
        mapped_means_test = df_test.set_index(['Store', 'Dept']).index.map(group_means[col])
        mapped_means_test_series = pd.Series(mapped_means_test, index=df_test.index)
        
        df_test[col] = df_test[col].fillna(mapped_means_test_series)
        df_test[col] = df_test[col].fillna(0) # Fallback para 0
    elif col in df_test.columns: # Se a coluna existe mas não há média do treino para ela
        print(f"AVISO: Não há média de treino para {col} (ou coluna não em group_means). Preenchendo NaNs em df_test com 0.")
        df_test[col] = df_test[col].fillna(0)
print("NaNs preenchidos em df_test.")

# Verificação final de NaNs
print("\n--- Verificando NaNs APÓS o tratamento final (V11) ---")
if 'df_train_subset' in locals() and cols_to_treat:
    print("\nFeatures em df_train_subset:")
    print(df_train_subset[cols_to_treat].isnull().sum())
if 'df_val_subset' in locals() and cols_to_treat:
    print("\nFeatures em df_val_subset:")
    print(df_val_subset[cols_to_treat].isnull().sum())
if 'df_test' in locals() and cols_to_treat:
    print("\nFeatures em df_test:")
    print(df_test[cols_to_treat].isnull().sum())

print("\nTratamento final de NaNs nas features de janela móvel e lag concluído (V11 - sem risco de leakage)!")

# Modelagem e Avaliação

## Divisão dos Dados (Definição das Variáveis (X e y))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Garantir que a coluna 'Date' esteja no formato datetime
df_train_merged['Date'] = pd.to_datetime(df_train_merged['Date'])
df_test_merged['Date'] = pd.to_datetime(df_test_merged['Date'])

# 2. Divisão dos dados em treino e validação a partir de df_train_merged
# Ordenar por data para garantir divisão temporal
df_train_merged = df_train_merged.sort_values('Date')

# Definir o ponto de corte para 80% dos dados
train_size = int(len(df_train_merged) * 0.8)
df_treino = df_train_merged.iloc[:train_size].copy()
df_validacao = df_train_merged.iloc[train_size:].copy()

# 3. Definir a variável alvo (y) para treino e validação
y_treino = df_treino['Weekly_Sales']
y_validacao = df_validacao['Weekly_Sales']

# 4. Definir as colunas de features
# Excluir 'Weekly_Sales' (alvo), mas manter 'Date' para gráficos futuros
colunas_features = [col for col in df_treino.columns if col != 'Weekly_Sales']

# Garantir que as mesmas colunas estejam em todos os conjuntos
X_treino = df_treino[colunas_features].copy()
X_validacao = df_validacao[colunas_features].copy()
X_teste = df_test_merged[colunas_features].copy()

# 5. Criar cópias de 'Date' para uso futuro em gráficos
date_treino = X_treino['Date'].copy()
date_validacao = X_validacao['Date'].copy()
date_teste = X_teste['Date'].copy()

# 6. Remover 'Date' das features para modelagem
colunas_features_modelagem = [col for col in colunas_features if col != 'Date']
X_treino = X_treino[colunas_features_modelagem]
X_validacao = X_validacao[colunas_features_modelagem]
X_teste = X_teste[colunas_features_modelagem]

# 7. Verificar os shapes
print("--- Shapes das Variáveis Definidas ---")
print(f"Shape de X_treino: {X_treino.shape}")
print(f"Shape de y_treino: {y_treino.shape}")
print(f"Shape de date_treino: {date_treino.shape}")
print(f"Shape de X_validacao: {X_validacao.shape}")
print(f"Shape de y_validacao: {y_validacao.shape}")
print(f"Shape de date_validacao: {date_validacao.shape}")
print(f"Shape de X_teste: {X_teste.shape}")
print(f"Shape de date_teste: {date_teste.shape}")

# 8. Verificar as colunas
print("\n--- Colunas em X_treino ---")
print(X_treino.columns.tolist())

# 9. Verificar consistência das colunas
if list(X_treino.columns) == list(X_validacao.columns) == list(X_teste.columns):
    print("\nAs colunas de features estão consistentes entre os conjuntos.")
else:
    print("\nATENÇÃO: As colunas de features NÃO estão consistentes entre os conjuntos!")
    print("Colunas em X_treino:", X_treino.columns.tolist())
    print("Colunas em X_validacao:", X_validacao.columns.tolist())
    print("Colunas em X_teste:", X_teste.columns.tolist())

# 10. Verificar intervalos de tempo
print("\n--- Intervalos de Tempo ---")
print(f"Treino - Data mínima: {date_treino.min()}, Data máxima: {date_treino.max()}")
print(f"Validação - Data mínima: {date_validacao.min()}, Data máxima: {date_validacao.max()}")
print(f"Teste - Data mínima: {date_teste.min()}, Data máxima: {date_teste.max()}")

#  Treinamento e Avaliação de Modelos

## Preparação dos dados para treino do modelo

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# # (Assumindo que df_train_subset, df_val_subset, e df_test estão definidos a partir de um carregamento anterior)
# # Exemplo: Carregamento de dados (substitua pelo seu método de carregamento real)
# # df_train_subset = pd.read_csv('train_subset.csv')
# # df_val_subset = pd.read_csv('val_subset.csv')

# print("\n--- Preparando Dados para Treinamento (X e y) ---")

# if 'df_train_subset' in locals() and 'df_val_subset' in locals():
#     # Garantir que 'Date' está em formato datetime
#     df_train_subset['Date'] = pd.to_datetime(df_train_subset['Date'])
#     df_val_subset['Date'] = pd.to_datetime(df_val_subset['Date'])

#     # Criar colunas temporais derivadas
#     df_train_subset['Year'] = df_train_subset['Date'].dt.year
#     df_train_subset['Month'] = df_train_subset['Date'].dt.month
#     df_train_subset['WeekOfYear'] = df_train_subset['Date'].dt.isocalendar().week
#     df_train_subset['DayOfYear'] = df_train_subset['Date'].dt.dayofyear
#     df_train_subset['Quarter'] = df_train_subset['Date'].dt.quarter

#     df_val_subset['Year'] = df_val_subset['Date'].dt.year
#     df_val_subset['Month'] = df_val_subset['Date'].dt.month
#     df_val_subset['WeekOfYear'] = df_val_subset['Date'].dt.isocalendar().week
#     df_val_subset['DayOfYear'] = df_val_subset['Date'].dt.dayofyear
#     df_val_subset['Quarter'] = df_val_subset['Date'].dt.quarter

#     # Definir as features, excluindo colunas indesejadas
#     features = [col for col in df_train_subset.columns 
#                 if col not in ['Store', 'Dept', 'Weekly_Sales', 'Date']
#                 and not pd.api.types.is_datetime64_any_dtype(df_train_subset[col])]

#     # Garantir que colunas temporais sejam incluídas
#     colunas_temporais = ['Year', 'Month', 'WeekOfYear', 'DayOfYear', 'Quarter']
#     features.extend([col for col in colunas_temporais if col in df_train_subset.columns and col not in features])

#     X_treino = df_train_subset[features]
#     y_treino = df_train_subset['Weekly_Sales']
#     X_validacao = df_val_subset[features]
#     y_validacao = df_val_subset['Weekly_Sales']

#     if 'df_test' in locals():
#         df_test['Date'] = pd.to_datetime(df_test['Date'])
#         df_test['Year'] = df_test['Date'].dt.year
#         df_test['Month'] = df_test['Date'].dt.month
#         df_test['WeekOfYear'] = df_test['Date'].dt.isocalendar().week
#         df_test['DayOfYear'] = df_test['Date'].dt.dayofyear
#         df_test['Quarter'] = df_test['Date'].dt.quarter
#         X_teste = df_test[features]
#         print(f"Shape de X_teste: {X_teste.shape}")
#     else:
#         print("AVISO: df_test não definido. X_teste não foi criado.")

#     print(f"Shape de X_treino: {X_treino.shape}, Shape de y_treino: {y_treino.shape}")
#     print(f"Shape de X_validacao: {X_validacao.shape}, Shape de y_validacao: {y_validacao.shape}")
#     print(f"\nPrimeiras 5 features selecionadas: {features[:5]}")
#     print(f"Total de features selecionadas: {len(features)}")
#     print("Colunas em X_validacao:", X_validacao.columns.tolist())
# else:
#     print("ERRO: df_train_subset ou df_val_subset não estão definidos. Interrompendo.")
#     X_treino, y_treino, X_validacao, y_validacao = [None] * 4

# # Treinar o modelo de regressão linear
# print("\n--- Treinando o Modelo de Regressão Linear ---")
# if X_treino is not None and y_treino is not None:
#     modelo_lr = LinearRegression()
#     modelo_lr.fit(X_treino, y_treino)

#     # Gerar previsões
#     y_pred_lr = modelo_lr.predict(X_validacao)
#     print("Modelo treinado e previsões geradas com sucesso.")
# else:
#     print("ERRO: X_treino ou y_treino não definidos. Não é possível treinar o modelo.")

# # Avaliar o modelo
# print("\n--- Avaliando o Modelo de Regressão Linear ---")
# if y_validacao is not None and y_pred_lr is not None:
#     # Calcular métricas
#     r2 = r2_score(y_validacao, y_pred_lr)
#     mae = mean_absolute_error(y_validacao, y_pred_lr)
#     mse = mean_squared_error(y_validacao, y_pred_lr)
#     rmse = np.sqrt(mse)  # Calcular RMSE manualmente
#     print(f"Métricas de Avaliação - Regressão Linear:")
#     print(f"R²: {r2:.4f}")
#     print(f"MAE: {mae:.2f}")
#     print(f"RMSE: {rmse:.2f}")
# else:
#     print("ERRO: y_validacao ou y_pred_lr não definidos. Não é possível avaliar o modelo.")

## Diagnóstico de dados antes do treinamento

In [ ]:
if X_treino is not None: # Prosseguir apenas se X_treino foi definido
    print("\n--- Diagnóstico de Dados Antes do Treinamento ---")
    print("\nTipos de dados em X_treino (Top 5):")
    print(X_treino.dtypes.head())
    print("\nVerificando NaN em X_treino (Soma Total):")
    print(X_treino.isnull().sum().sum()) # Mostra a soma total de NaNs
    print("\nVerificando NaN em y_treino:")
    print(y_treino.isnull().sum())
    print("\nVerificando NaN em X_validacao (Soma Total):")
    print(X_validacao.isnull().sum().sum())
    print("\nVerificando NaN em y_validacao:")
    print(y_validacao.isnull().sum())
    
    print("\nVerificando valores infinitos em X_treino (Soma Total):")
    print(np.isinf(X_treino.select_dtypes(include=np.number)).sum().sum()) # Apenas em colunas numéricas
    print("\nVerificando valores infinitos em y_treino:")
    print(np.isinf(y_treino).sum())
    print("\nVerificando valores infinitos em X_validacao (Soma Total):")
    print(np.isinf(X_validacao.select_dtypes(include=np.number)).sum().sum())
    print("\nVerificando valores infinitos em y_validacao:")
    print(np.isinf(y_validacao).sum())
    print("\nDiagnóstico concluído.")
else:
    print("\nDiagnóstico de dados não pode ser realizado pois X_treino não foi definido.")

## Treinamento do modelo

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import psutil  # Para monitorar uso de memória (opcional)

print("\n--- Iniciando Comparação de Modelos (Treino e Validação) ---")

# Função para monitorar uso de memória (opcional)
def check_memory_usage():
    process = psutil.Process()
    memory_info = process.memory_info()
    print(f"Uso de memória atual: {memory_info.rss / 1024 / 1024:.2f} MB")

# Recriar X_treino, y_treino, X_validacao e y_validacao se necessário
if 'X_treino' not in locals() or 'y_treino' not in locals() or 'X_validacao' not in locals() or 'y_validacao' not in locals():
    print("\nAVISO: X_treino, y_treino, X_validacao ou y_validacao não estão definidos. Recriando a partir de df_train_merged...")
    if 'df_train_merged' not in locals():
        print("ERRO: df_train_merged não está definido. Não é possível recriar os dados.")
    else:
        # Divisão temporal 80/20
        df_train_merged = df_train_merged.sort_values('Date')
        train_size = int(len(df_train_merged) * 0.8)
        df_treino = df_train_merged.iloc[:train_size].copy()
        df_validacao = df_train_merged.iloc[train_size:].copy()
        y_treino = df_treino['Weekly_Sales'].copy()
        y_validacao = df_validacao['Weekly_Sales'].copy()
        colunas_a_excluir = ['Weekly_Sales', 'Date']
        colunas_features = [col for col in df_treino.columns if col not in colunas_a_excluir]
        colunas_features = [f for f in colunas_features if not pd.api.types.is_datetime64_any_dtype(df_treino[f])]
        X_treino = df_treino[colunas_features].copy()
        X_validacao = df_validacao[colunas_features].copy()
        print(f"Dados recriados com sucesso. Shape de X_treino: {X_treino.shape}, Shape de X_validacao: {X_validacao.shape}")

# Tratamento de NaNs e Infinitos antes de qualquer manipulação
if 'X_treino' in locals() and 'X_validacao' in locals():
    for df_X, nome_df in zip([X_treino, X_validacao], ['X_treino', 'X_validacao']):
        print(f"\n--- Verificando e Tratando NaNs e Infinitos em {nome_df} ---")
        # Verificar NaNs
        nan_check = df_X.isnull().sum()
        colunas_com_nan = nan_check[nan_check > 0]
        if not colunas_com_nan.empty:
            print(f"Colunas com NaNs em {nome_df} antes do preenchimento:")
            print(colunas_com_nan)
            print(f"Preenchendo NaNs em {nome_df}...")
            cpi_fill_value = X_treino['CPI'].median() if 'CPI' in X_treino.columns else 0
            unemp_fill_value = X_treino['Unemployment'].median() if 'Unemployment' in X_treino.columns else 0
            for col_nan in colunas_com_nan.index:
                if col_nan == 'CPI':
                    df_X[col_nan] = df_X[col_nan].fillna(cpi_fill_value)
                elif col_nan == 'Unemployment':
                    df_X[col_nan] = df_X[col_nan].fillna(unemp_fill_value)
                else:
                    df_X[col_nan] = df_X[col_nan].fillna(0)
            print(f"NaNs preenchidos em {nome_df}. Nova verificação:")
            print(df_X.isnull().sum()[df_X.isnull().sum() > 0])
        else:
            print(f"Nenhum NaN encontrado em {nome_df}.")
        
        # Verificar infinitos sem criar cópias desnecessárias
        numeric_cols = df_X.select_dtypes(include=np.number).columns
        inf_check = pd.Series(0, index=numeric_cols)  # Inicializa com zeros
        for col in numeric_cols:
            inf_check[col] = np.isinf(df_X[col]).sum()
        colunas_com_inf = inf_check[inf_check > 0]
        if not colunas_com_inf.empty:
            print(f"Colunas com infinitos em {nome_df}:")
            print(colunas_com_inf)
            print(f"Substituindo infinitos por NaN e preenchendo com 0 em {nome_df}...")
            df_X[numeric_cols] = df_X[numeric_cols].replace([np.inf, -np.inf], np.nan)
            df_X[numeric_cols] = df_X[numeric_cols].fillna(0)
            print(f"Infinitos tratados em {nome_df}.")

# Codificação da coluna 'Type' após tratamento de NaNs
if 'X_treino' in locals() and 'X_validacao' in locals():
    print("\n--- Codificando a Coluna 'Type' ---")
    type_mapping = {'A': 3, 'B': 2, 'C': 1}
    if 'Type' in X_treino.columns:
        try:
            X_treino['Type'] = X_treino['Type'].map(type_mapping).astype('int32')
            X_validacao['Type'] = X_validacao['Type'].map(type_mapping).astype('int32')
            print("Valores únicos em 'Type' após mapeamento (X_treino):", X_treino['Type'].unique())
            print("Valores únicos em 'Type' após mapeamento (X_validacao):", X_validacao['Type'].unique())
        except Exception as e:
            print(f"ERRO ao codificar 'Type': {e}")
    else:
        print("AVISO: Coluna 'Type' não encontrada em X_treino ou X_validacao.")

    # Verificar se há outras colunas categóricas
    print("\n--- Verificando Tipos de Dados para Identificar Colunas Categóricas ---")
    colunas_nao_numericas = X_treino.select_dtypes(include=['object', 'category']).columns.tolist()
    if colunas_nao_numericas:
        print(f"Colunas não numéricas encontradas em X_treino: {colunas_nao_numericas}")
        print("ERRO: Existem colunas categóricas que precisam ser codificadas antes de prosseguir.")
        X_treino_scaled, X_validacao_scaled = None, None
    else:
        print("Nenhuma coluna categórica adicional encontrada.")

# Diagnóstico de Dados (após tratamento e codificação)
if X_treino is not None:
    print("\n--- Diagnóstico de Dados Após Tratamento ---")
    print("\nTipos de dados em X_treino (Top 5):")
    print(X_treino.dtypes.head())
    print("\nVerificando NaN em X_treino (Soma Total):")
    print(X_treino.isnull().sum().sum())
    print("\nVerificando NaN em y_treino:")
    print(y_treino.isnull().sum())
    print("\nVerificando NaN em X_validacao (Soma Total):")
    print(X_validacao.isnull().sum().sum())
    print("\nVerificando NaN em y_validacao:")
    print(y_validacao.isnull().sum())
    print("\nVerificando valores infinitos em X_treino (Soma Total):")
    print(np.isinf(X_treino.select_dtypes(include=np.number)).sum().sum())
    print("\nVerificando valores infinitos em y_treino:")
    print(np.isinf(y_treino).sum())
    print("\nVerificando valores infinitos em X_validacao (Soma Total):")
    print(np.isinf(X_validacao.select_dtypes(include=np.number)).sum().sum())
    print("\nVerificando valores infinitos em y_validacao:")
    print(np.isinf(y_validacao).sum())
    print("\nDiagnóstico concluído.")

# Converter para tipos de dados otimizados para reduzir uso de memória
if 'X_treino' in locals() and 'X_validacao' in locals() and colunas_nao_numericas == []:
    print("\n--- Otimizando Tipos de Dados para Reduzir Uso de Memória ---")
    for df_X, nome_df in zip([X_treino, X_validacao], ['X_treino', 'X_validacao']):
        for col in df_X.columns:
            if df_X[col].dtype == 'int64':
                df_X[col] = df_X[col].astype('int32')
            elif df_X[col].dtype == 'float64':
                df_X[col] = df_X[col].astype('float32')
            elif df_X[col].dtype == 'bool':
                df_X[col] = df_X[col].astype('bool')  # Já é otimizado, mas para consistência
        print(f"Tipos de dados otimizados em {nome_df}:")
        print(df_X.dtypes.head())

# Escalonar os dados
if 'X_treino' in locals() and 'X_validacao' in locals() and colunas_nao_numericas == []:
    check_memory_usage()  # Monitorar uso de memória antes do escalonamento
    if 'scaler' not in locals():
        print("\nAVISO: Scaler não encontrado. Criando um novo scaler a partir de X_treino...")
        scaler = StandardScaler()
        X_treino_scaled = scaler.fit_transform(X_treino.to_numpy().astype(np.float32))
        X_validacao_scaled = scaler.transform(X_validacao.to_numpy().astype(np.float32))
    else:
        print("\nReutilizando scaler existente...")
        X_treino_scaled = scaler.transform(X_treino.to_numpy().astype(np.float32))
        X_validacao_scaled = scaler.transform(X_validacao.to_numpy().astype(np.float32))
    print("Escalonamento concluído.")
    check_memory_usage()  # Monitorar uso de memória após o escalonamento
else:
    print("\nERRO: Dados de treino ou validação não estão definidos corretamente ou contêm colunas categóricas.")
    X_treino_scaled, X_validacao_scaled = None, None

# Treinamento e Avaliação dos Modelos
if X_treino_scaled is not None and y_treino is not None and X_validacao_scaled is not None and y_validacao is not None:
    resultados_modelos = {}

    # --- Modelo 1: Regressão Linear (Baseline) ---
    print("\nTreinando Modelo: Regressão Linear...")
    lr_model = LinearRegression()
    lr_model.fit(X_treino_scaled, y_treino)
    y_pred_lr = lr_model.predict(X_validacao_scaled)
    mae_lr = mean_absolute_error(y_validacao, y_pred_lr)
    rmse_lr = np.sqrt(mean_squared_error(y_validacao, y_pred_lr))
    r2_lr = r2_score(y_validacao, y_pred_lr)
    print(f"Regressão Linear - MAE: {mae_lr:.2f}, RMSE: {rmse_lr:.2f}, R²: {r2_lr:.4f}")
    resultados_modelos['Regressão Linear'] = {'MAE': mae_lr, 'RMSE': rmse_lr, 'R2': r2_lr}

    # --- Modelo 2: Árvore de Decisão Regressora ---
    print("\nTreinando Modelo: Árvore de Decisão Regressora...")
    dt_model = DecisionTreeRegressor(max_depth=10, min_samples_split=10, min_samples_leaf=5, random_state=42)
    dt_model.fit(X_treino, y_treino)  # Árvores não requerem escalonamento
    y_pred_dt = dt_model.predict(X_validacao)
    mae_dt = mean_absolute_error(y_validacao, y_pred_dt)
    rmse_dt = np.sqrt(mean_squared_error(y_validacao, y_pred_dt))
    r2_dt = r2_score(y_validacao, y_pred_dt)
    print(f"Árvore de Decisão - MAE: {mae_dt:.2f}, RMSE: {rmse_dt:.2f}, R²: {r2_dt:.4f}")
    resultados_modelos['Árvore de Decisão'] = {'MAE': mae_dt, 'RMSE': rmse_dt, 'R2': r2_dt}

    # --- Modelo 3: Random Forest Regressor ---
    print("\nTreinando Modelo: Random Forest Regressor...")
    rf_model = RandomForestRegressor(n_estimators=100, max_depth=20, min_samples_split=5, min_samples_leaf=5, random_state=42, n_jobs=-1)
    rf_model.fit(X_treino, y_treino)  # Árvores não requerem escalonamento
    y_pred_rf = rf_model.predict(X_validacao)
    mae_rf = mean_absolute_error(y_validacao, y_pred_rf)
    rmse_rf = np.sqrt(mean_squared_error(y_validacao, y_pred_rf))
    r2_rf = r2_score(y_validacao, y_pred_rf)
    print(f"Random Forest - MAE: {mae_rf:.2f}, RMSE: {rmse_rf:.2f}, R²: {r2_rf:.4f}")
    resultados_modelos['Random Forest'] = {'MAE': mae_rf, 'RMSE': rmse_rf, 'R2': r2_rf}

    # --- Modelo 4: Gradient Boosting Regressor ---
    print("\nTreinando Modelo: Gradient Boosting Regressor...")
    gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, min_samples_split=5, min_samples_leaf=5, random_state=42)
    gb_model.fit(X_treino, y_treino)  # Árvores não requerem escalonamento
    y_pred_gb = gb_model.predict(X_validacao)
    mae_gb = mean_absolute_error(y_validacao, y_pred_gb)
    rmse_gb = np.sqrt(mean_squared_error(y_validacao, y_pred_gb))
    r2_gb = r2_score(y_validacao, y_pred_gb)
    print(f"Gradient Boosting - MAE: {mae_gb:.2f}, RMSE: {rmse_gb:.2f}, R²: {r2_gb:.4f}")
    resultados_modelos['Gradient Boosting'] = {'MAE': mae_gb, 'RMSE': rmse_gb, 'R2': r2_gb}

    # --- Resultados Consolidados ---
    print("\n--- Resultados Consolidados da Avaliação (no conjunto de validação) ---")
    df_resultados = pd.DataFrame(resultados_modelos).T
    print(df_resultados.sort_values(by='RMSE'))
else:
    print("\nModelos não foram treinados pois os dados de treino/validação (X, y) não estão definidos corretamente.")

In [ ]:
print("Média de Weekly_Sales no treino:", y_treino.mean())
print("Desvio padrão de Weekly_Sales no treino:", y_treino.std())
print("Média de Weekly_Sales na validação:", y_validacao.mean())
print("Desvio padrão de Weekly_Sales na validação:", y_validacao.std())

## Ajuste do Código para Análise de Importância das Features (Regressão Linear)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

print("--- Análise de Importância das Features (Regressão Linear) ---")

# 1. Verificar se o modelo de Regressão Linear está treinado
if 'lr_model' not in locals() or lr_model is None:
    print("Modelo de Regressão Linear não encontrado. Treinando um novo modelo...")
    lr_model = LinearRegression()
    lr_model.fit(X_treino, y_treino)  # Assumindo que X_treino e y_treino estão definidos

# 2. Verificar se X_treino está definido
if 'X_treino' not in locals() or X_treino is None:
    print("ERRO: X_treino não está definido. Não é possível prosseguir com a análise de importância.")
else:
    # 3. Extrair os coeficientes do modelo de Regressão Linear
    coeficientes = lr_model.coef_
    feature_names = X_treino.columns

    if len(coeficientes) == len(feature_names):
        # 4. Criar um DataFrame para os coeficientes
        feature_importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Coefficient': coeficientes,
            'Abs_Coefficient': np.abs(coeficientes)  # Usar valor absoluto para comparar importância
        })

        # 5. Ordenar as features pela importância (valor absoluto dos coeficientes)
        feature_importance_df = feature_importance_df.sort_values(by='Abs_Coefficient', ascending=False)

        # 6. Exibir as 15 features mais importantes
        print("\nImportância das Features (Top 15, Regressão Linear):")
        print(feature_importance_df[['Feature', 'Coefficient', 'Abs_Coefficient']].head(15))

        # 7. Plotar a importância das features (Top 20)
        plt.figure(figsize=(10, 8))
        top_n_features = 20
        sns.barplot(
            x='Abs_Coefficient',
            y='Feature',
            data=feature_importance_df.head(top_n_features),
            hue='Feature',
            palette='viridis',
            legend=False
        )
        plt.title(f'Top {top_n_features} Features Mais Importantes (Regressão Linear)')
        plt.xlabel('Magnitude do Coeficiente (Importância)')
        plt.ylabel('Feature')
        plt.tight_layout()
        plt.show()
    else:
        print("ERRO: O número de coeficientes não corresponde ao número de nomes de features.")
        print(f"Coeficientes: {len(coeficientes)}, Nomes de Features: {len(feature_names)}")

In [ ]:
# Adicionar esta seção após o treinamento dos modelos, mas antes dos resultados consolidados

# Avaliação no Conjunto de Treino
resultados_treino = {}

# Regressão Linear
y_pred_lr_treino = lr_model.predict(X_treino_scaled)
mae_lr_treino = mean_absolute_error(y_treino, y_pred_lr_treino)
rmse_lr_treino = np.sqrt(mean_squared_error(y_treino, y_pred_lr_treino))
r2_lr_treino = r2_score(y_treino, y_pred_lr_treino)
resultados_treino['Regressão Linear'] = {'MAE': mae_lr_treino, 'RMSE': rmse_lr_treino, 'R2': r2_lr_treino}

# Árvore de Decisão
y_pred_dt_treino = dt_model.predict(X_treino)
mae_dt_treino = mean_absolute_error(y_treino, y_pred_dt_treino)
rmse_dt_treino = np.sqrt(mean_squared_error(y_treino, y_pred_dt_treino))
r2_dt_treino = r2_score(y_treino, y_pred_dt_treino)
resultados_treino['Árvore de Decisão'] = {'MAE': mae_dt_treino, 'RMSE': rmse_dt_treino, 'R2': r2_dt_treino}

# Random Forest
y_pred_rf_treino = rf_model.predict(X_treino)
mae_rf_treino = mean_absolute_error(y_treino, y_pred_rf_treino)
rmse_rf_treino = np.sqrt(mean_squared_error(y_treino, y_pred_rf_treino))
r2_rf_treino = r2_score(y_treino, y_pred_rf_treino)
resultados_treino['Random Forest'] = {'MAE': mae_rf_treino, 'RMSE': rmse_rf_treino, 'R2': r2_rf_treino}

# Gradient Boosting
y_pred_gb_treino = gb_model.predict(X_treino)
mae_gb_treino = mean_absolute_error(y_treino, y_pred_gb_treino)
rmse_gb_treino = np.sqrt(mean_squared_error(y_treino, y_pred_gb_treino))
r2_gb_treino = r2_score(y_treino, y_pred_gb_treino)
resultados_treino['Gradient Boosting'] = {'MAE': mae_gb_treino, 'RMSE': rmse_gb_treino, 'R2': r2_gb_treino}

# Exibir resultados no conjunto de treino
print("\n--- Resultados Consolidados da Avaliação (no conjunto de treino) ---")
df_resultados_treino = pd.DataFrame(resultados_treino).T
print(df_resultados_treino.sort_values(by='RMSE'))

# Comparar com os resultados no conjunto de validação
print("\n--- Comparação Treino vs Validação (R²) ---")
comparacao_r2 = pd.DataFrame({
    'R2_Treino': df_resultados_treino['R2'],
    'R2_Validação': df_resultados['R2'],
    'Diferença': df_resultados_treino['R2'] - df_resultados['R2']
})
print(comparacao_r2)

## Treinamento do Modelo Final (RandomForestRegressor v2) e Previsões no Conjunto de Teste

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

print("--- Treinamento do Modelo Final e Previsões no Teste ---")

# 1: Definir as colunas de features e preparar X_full_train, y_full_train, X_teste
print("\n--- Preparando Dados para Treinamento Final ---")

if 'df_train_merged' not in locals() or 'df_test_merged' not in locals():
    print("ERRO: df_train_merged ou df_test_merged não estão definidos. Interrompendo.")
    X_full_train, y_full_train, X_teste = None, None, None
else:
    # Codificação da coluna 'Type'
    print("\n--- Codificando a Coluna 'Type' ---")
    type_mapping = {'A': 3, 'B': 2, 'C': 1}
    df_train_merged['Type'] = df_train_merged['Type'].map(type_mapping).astype('int32')
    df_test_merged['Type'] = df_test_merged['Type'].map(type_mapping).astype('int32')

    # Verificar se há outros valores não mapeados
    print("Valores únicos em 'Type' após mapeamento (df_train_merged):", df_train_merged['Type'].unique())
    print("Valores únicos em 'Type' após mapeamento (df_test_merged):", df_test_merged['Type'].unique())

    colunas_a_excluir = ['Weekly_Sales', 'Date']
    
    colunas_features = [col for col in df_train_merged.columns if col not in colunas_a_excluir]
    colunas_features = [f for f in colunas_features if not pd.api.types.is_datetime64_any_dtype(df_train_merged[f])]

    print(f"Total de features selecionadas: {len(colunas_features)}")
    if len(colunas_features) < 5:
        print(f"AVISO: Poucas features selecionadas ({len(colunas_features)}). Verifique 'colunas_features'. Features: {colunas_features}")

    X_full_train = df_train_merged[colunas_features].copy()
    y_full_train = df_train_merged['Weekly_Sales'].copy()
    
    # Preparar X_teste, garantindo consistência de colunas com X_full_train
    X_teste = df_test_merged.copy()
    
    # Adicionar colunas faltantes em X_teste com valor 0
    for col in X_full_train.columns:
        if col not in X_teste.columns:
            print(f"Adicionando coluna faltante '{col}' em X_teste com valor 0.")
            X_teste[col] = 0
            
    # Garantir que X_teste tenha apenas as colunas de X_full_train e na mesma ordem
    try:
        X_teste = X_teste[X_full_train.columns]
        print("Colunas de X_teste alinhadas com X_full_train.")
    except KeyError as e:
        print(f"ERRO ao alinhar colunas de X_teste: {e}")
        print("Algumas colunas esperadas em X_full_train não puderam ser encontradas ou criadas em X_teste.")
        print(f"Colunas em X_full_train: {list(X_full_train.columns)}")
        print(f"Colunas em X_teste antes do alinhamento: {list(df_test_merged.copy().columns)}")
        X_teste = None

    if X_teste is not None:
        print(f"\nShape de X_full_train: {X_full_train.shape}")
        print(f"Shape de y_full_train: {y_full_train.shape}")
        print(f"Shape de X_teste: {X_teste.shape}")
    else:
        print("\nFalha na preparação de X_teste.")

# 2: Verificação e Tratamento de NaNs em X_full_train e X_teste
if X_full_train is not None and X_teste is not None:
    print("\n--- Verificando e Tratando NaNs Remanescentes ---")
    # Calcular medianas do TREINO para CPI e Unemployment
    cpi_fill_value = X_full_train['CPI'].median() if 'CPI' in X_full_train.columns else 0
    unemp_fill_value = X_full_train['Unemployment'].median() if 'Unemployment' in X_full_train.columns else 0

    for df_X, nome_df in zip([X_full_train, X_teste], ['X_full_train', 'X_teste']):
        print(f"\nProcessando NaNs em {nome_df}:")
        nan_check = df_X.isnull().sum()
        colunas_com_nan = nan_check[nan_check > 0]
        
        if not colunas_com_nan.empty:
            print(f"Colunas com NaNs em {nome_df} antes do preenchimento:")
            print(colunas_com_nan)
            print(f"Preenchendo NaNs restantes em {nome_df}...")
            for col_nan in colunas_com_nan.index:
                if col_nan == 'CPI':
                    df_X[col_nan] = df_X[col_nan].fillna(cpi_fill_value)
                elif col_nan == 'Unemployment':
                    df_X[col_nan] = df_X[col_nan].fillna(unemp_fill_value)
                else:
                    df_X[col_nan] = df_X[col_nan].fillna(0)
            print(f"NaNs preenchidos em {nome_df}. Nova verificação:")
            print(df_X.isnull().sum()[df_X.isnull().sum() > 0])
        else:
            print(f"Nenhum NaN encontrado em {nome_df}.")

# 3: Verificar Infinitos
if X_full_train is not None and X_teste is not None:
    print("\n--- Verificando Infinitos ---")
    # Evitar cópias desnecessárias ao verificar infinitos
    numeric_cols_train = X_full_train.select_dtypes(include=np.number).columns
    inf_check_train = pd.Series(0, index=numeric_cols_train)
    for col in numeric_cols_train:
        inf_check_train[col] = np.isinf(X_full_train[col]).sum()
    print(f"Infinitos em X_full_train: {inf_check_train.sum()}")

    numeric_cols_test = X_teste.select_dtypes(include=np.number).columns
    inf_check_test = pd.Series(0, index=numeric_cols_test)
    for col in numeric_cols_test:
        inf_check_test[col] = np.isinf(X_teste[col]).sum()
    print(f"Infinitos em X_teste: {inf_check_test.sum()}")

# 4: Otimizar Tipos de Dados para Reduzir Uso de Memória
if X_full_train is not None and X_teste is not None:
    print("\n--- Otimizando Tipos de Dados para Reduzir Uso de Memória ---")
    for df_X, nome_df in zip([X_full_train, X_teste], ['X_full_train', 'X_teste']):
        for col in df_X.columns:
            if df_X[col].dtype == 'int64':
                df_X[col] = df_X[col].astype('int32')
            elif df_X[col].dtype == 'float64':
                df_X[col] = df_X[col].astype('float32')
            elif df_X[col].dtype == 'bool':
                df_X[col] = df_X[col].astype('bool')  # Já otimizado, mas para consistência
        print(f"Tipos de dados otimizados em {nome_df}:")
        print(df_X.dtypes.head())

# 5: Treinar o Modelo Final (Random Forest)
if X_full_train is not None and y_full_train is not None and X_teste is not None:
    print("\n--- Treinando o Modelo Final ---")
    
    print("Modelo Escolhido: Random Forest Regressor")
    final_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1  # Usa todos os núcleos disponíveis para acelerar o treinamento
    )
    final_model.fit(X_full_train, y_full_train)
    print("Treinamento do modelo final concluído.")

    # 6: Fazer Previsões no Conjunto de Teste
    print("\n--- Fazendo Previsões no Conjunto de Teste ---")
    predicoes_finais_teste = final_model.predict(X_teste)
    print("Previsões no conjunto de teste concluídas.")
    print(f"Número de previsões geradas: {len(predicoes_finais_teste)}")
    print("Exemplo das primeiras 5 previsões:", predicoes_finais_teste[:5])

    # 7: Opcional - Importância das Features
    print("\n--- Importância das Features ---")
    importancias = pd.DataFrame({
        'Feature': X_full_train.columns,
        'Importância': final_model.feature_importances_
    })
    print(importancias.sort_values(by='Importância', ascending=False).head(10))
else:
    print("\nTreinamento do modelo e previsões não podem ser realizados devido a erros na preparação dos dados.")

In [ ]:
condicao_de_parada = True # Defina sua condição
if condicao_de_parada:
    print("Parando a execução aqui.")
    raise SystemExit("Execução interrompida propositalmente.")
print("Isso não será executado se a condição acima for verdadeira.")

# Previsões no teste (Validação do modelo)

## Permutation Importance 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

print("\n--- Calculando Importância por Permutação (Regressão Linear) ---")

# Verifique se as variáveis necessárias existem
if 'final_model' not in locals():
    print("ERRO: Modelo 'final_model' não está definido. Interrompendo.")
elif 'X_validacao' not in locals() or 'y_validacao' not in locals():
    print("AVISO: X_validacao ou y_validacao não estão definidos. Recriando a partir de df_train_merged...")
    if 'df_train_merged' not in locals():
        print("ERRO: df_train_merged não está definido. Não é possível recriar X_validacao e y_validacao.")
    else:
        # Recriar X_validacao e y_validacao com divisão temporal (80/20)
        df_train_merged = df_train_merged.sort_values('Date')
        train_size = int(len(df_train_merged) * 0.8)
        df_validacao = df_train_merged.iloc[train_size:].copy()
        y_validacao = df_validacao['Weekly_Sales'].copy()
        colunas_a_excluir = ['Weekly_Sales', 'Date']
        colunas_features = [col for col in df_validacao.columns if col not in colunas_a_excluir]
        colunas_features = [f for f in colunas_features if not pd.api.types.is_datetime64_any_dtype(df_validacao[f])]
        X_validacao = df_validacao[colunas_features].copy()
        print(f"X_validacao e y_validacao recriados com sucesso. Shape de X_validacao: {X_validacao.shape}")

# Verificar e tratar NaNs em X_validacao (se necessário)
if 'X_validacao' in locals() and X_validacao is not None:
    print("\n--- Verificando e Tratando NaNs em X_validacao ---")
    nan_check = X_validacao.isnull().sum()
    colunas_com_nan = nan_check[nan_check > 0]
    if not colunas_com_nan.empty:
        print("Colunas com NaNs em X_validacao antes do preenchimento:")
        print(colunas_com_nan)
        print("Preenchendo NaNs em X_validacao...")
        # Usar medianas de X_full_train para preenchimento (se disponível)
        if 'X_full_train' in locals():
            cpi_fill_value = X_full_train['CPI'].median() if 'CPI' in X_full_train.columns else 0
            unemp_fill_value = X_full_train['Unemployment'].median() if 'Unemployment' in X_full_train.columns else 0
            for col_nan in colunas_com_nan.index:
                if col_nan == 'CPI':
                    X_validacao[col_nan] = X_validacao[col_nan].fillna(cpi_fill_value)
                elif col_nan == 'Unemployment':
                    X_validacao[col_nan] = X_validacao[col_nan].fillna(unemp_fill_value)
                else:
                    X_validacao[col_nan] = X_validacao[col_nan].fillna(0)
        else:
            X_validacao = X_validacao.fillna(0)
        print("NaNs preenchidos em X_validacao. Nova verificação:")
        print(X_validacao.isnull().sum()[X_validacao.isnull().sum() > 0])
    else:
        print("Nenhum NaN encontrado em X_validacao.")

# Verificar se X_validacao_scaled existe; se não, criar com StandardScaler
if 'X_validacao_scaled' not in locals() or X_validacao_scaled is None:
    print("\nAVISO: X_validacao_scaled não está definido. Escalonando X_validacao...")
    if 'scaler' not in locals():
        print("AVISO: Scaler não encontrado. Criando um novo scaler a partir de X_full_train...")
        if 'X_full_train' not in locals():
            print("ERRO: X_full_train não está definido. Não é possível escalonar X_validacao.")
        else:
            scaler = StandardScaler()
            scaler.fit(X_full_train)  # Ajustar o scaler com X_full_train
            X_validacao_scaled = scaler.transform(X_validacao)  # Escalonar X_validacao
            print("Escalonamento de X_validacao concluído.")
    else:
        X_validacao_scaled = scaler.transform(X_validacao)  # Usar o scaler existente
        print("Escalonamento de X_validacao concluído com scaler existente.")

# Calcular a importância por permutação no conjunto de validação (escalonado)
if 'X_validacao_scaled' in locals() and X_validacao_scaled is not None:
    try:
        perm_importance = permutation_importance(
            final_model, 
            X_validacao_scaled,  # Usar X_validacao escalonado
            y_validacao, 
            n_repeats=5, 
            random_state=42, 
            n_jobs=-1,
            scoring='r2'  # Usar R² como métrica para consistência com regressão
        )

        # Organizar os resultados em um DataFrame
        sorted_idx = perm_importance.importances_mean.argsort()  # Índices ordenados

        perm_importance_df = pd.DataFrame(
            data=perm_importance.importances_mean[sorted_idx],
            index=X_validacao.columns[sorted_idx],  # Nomes das features na ordem correta
            columns=['Importance_Permutation']
        ).sort_values(by='Importance_Permutation', ascending=False)

        # Exibir as 15 features mais importantes
        print("\nImportância das Features por Permutação (Top 15, Regressão Linear):")
        print(perm_importance_df.head(15))

        # Plotar as 20 features mais importantes
        plt.figure(figsize=(10, 8))
        sns.barplot(
            x='Importance_Permutation',
            y=perm_importance_df.head(20).index,
            data=perm_importance_df.head(20),
            palette='viridis'
        )
        plt.title("Top 20 Features Mais Importantes (Permutation Importance - Regressão Linear)")
        plt.xlabel("Queda Média na Performance (R²)")
        plt.ylabel("Feature")
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"ERRO ao calcular permutation importance: {e}")
else:
    print("ERRO: X_validacao_scaled não está definido. Não é possível calcular permutation importance.")

## Partial Dependence Plots (PDP)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
from sklearn.preprocessing import StandardScaler

print("\n--- Gerando Partial Dependence Plots (Regressão Linear) ---")

# Verifique se as variáveis necessárias existem
if 'final_model' not in locals() or 'X_full_train' not in locals() or not isinstance(X_full_train, pd.DataFrame):
    print("ERRO: Variáveis 'final_model' ou 'X_full_train' não estão definidas ou X_full_train não é um DataFrame. Interrompendo.")
else:
    # Exibir colunas disponíveis para depuração
    print(f"Colunas disponíveis em X_full_train: {X_full_train.columns.tolist()}")

    # Escalonar X_full_train, mantendo nomes das colunas
    if 'X_full_train_scaled' not in locals() or X_full_train_scaled is None:
        print("AVISO: X_full_train_scaled não está definido. Escalonando X_full_train...")
        if 'scaler' not in locals():
            print("AVISO: Scaler não encontrado. Criando um novo scaler...")
            scaler = StandardScaler()
            X_full_train_scaled = scaler.fit_transform(X_full_train)
        else:
            print("Reutilizando scaler existente...")
            X_full_train_scaled = scaler.transform(X_full_train)
        # Converter X_full_train_scaled para DataFrame para preservar nomes das colunas
        X_full_train_scaled = pd.DataFrame(X_full_train_scaled, columns=X_full_train.columns)
        print("Escalonamento de X_full_train concluído.")

    # Escolha features para PDP com base nas mais importantes (ajustadas para colunas reais)
    features_para_pdp = [
        'IsHoliday',    # Já existe
        'Size',         # Já existe
        'CPI',          # Já existe
        'Weekly_Sales_lag_1_x',  # Substitui roll_mean_4 por lag_1 como proxy
        'Weekly_Sales_lag_4_x',  # Substitui roll_std_4 por lag_4 como proxy
        'Weekly_Sales_lag_52_x'  # Substitui roll_mean_52 por lag_52
    ]
    
    # Filtrar para garantir que apenas features existentes em X_full_train sejam usadas
    features_para_pdp_existentes = [f for f in features_para_pdp if f in X_full_train.columns]
    
    if not features_para_pdp_existentes:
        print("ERRO: Nenhuma das features selecionadas para PDP foi encontrada em X_full_train.")
        print(f"Colunas disponíveis: {X_full_train.columns.tolist()}")
    else:
        print(f"Gerando PDPs para: {features_para_pdp_existentes}")
        
        # Determinar o número de linhas e colunas para os subplots
        n_features_pdp = len(features_para_pdp_existentes)
        n_cols_pdp = 2 if n_features_pdp > 1 else 1
        n_rows_pdp = (n_features_pdp + n_cols_pdp - 1) // n_cols_pdp
        
        # Ajustar o tamanho da figura dinamicamente
        fig_height = 4 * n_rows_pdp
        fig_width = 6 * n_cols_pdp
        
        fig, ax = plt.subplots(n_rows_pdp, n_cols_pdp, figsize=(fig_width, fig_height), squeeze=False)
        ax = ax.flatten()

        # Gerar PDP para cada feature selecionada
        try:
            display_pdp = PartialDependenceDisplay.from_estimator(
                final_model,
                X_full_train_scaled,  # Usar dados escalonados com nomes de colunas
                features=features_para_pdp_existentes,
                kind='average',
                n_jobs=-1,
                grid_resolution=20,
                ax=ax[:n_features_pdp],
                feature_names=X_full_train.columns.tolist()  # Explicitamente passar nomes das colunas
            )
            
            # Remover eixos não utilizados
            for i in range(n_features_pdp, len(ax)):
                fig.delaxes(ax[i])
            
            # Ajustar título e layout
            plt.suptitle("Partial Dependence Plots (PDP) para Features Selecionadas (Regressão Linear)", fontsize=16, y=1.01)
            plt.tight_layout(rect=[0, 0, 1, 0.97])
            plt.show()
        except Exception as e:
            print(f"ERRO ao gerar os PDPs: {e}")
            print("Verifique se as features em 'features_para_pdp_existentes' são adequadas para PDP.")
            print(f"Features selecionadas: {features_para_pdp_existentes}")
            print(f"Colunas disponíveis em X_full_train: {X_full_train.columns.tolist()}")

## Avaliar Overfitting

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

print("--- Avaliando Overfitting: Performance no Treino vs. Validação ---")

# Dicionário para armazenar os resultados
resultados_treino = {}
resultados_validacao = {}

# Função auxiliar para calcular e imprimir métricas
def calcular_metricas(nome_modelo, modelo, X, y_real, sufixo_dataset=""):
    y_pred = modelo.predict(X)
    mae = mean_absolute_error(y_real, y_pred)
    rmse = np.sqrt(mean_squared_error(y_real, y_pred))
    r2 = r2_score(y_real, y_pred)
    print(f"{nome_modelo} - {sufixo_dataset} - MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.4f}")
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

# Recriar X_treino, y_treino, X_validacao e y_validacao se necessário
if 'X_treino' not in locals() or 'y_treino' not in locals() or 'X_validacao' not in locals() or 'y_validacao' not in locals():
    print("\nAVISO: X_treino, y_treino, X_validacao ou y_validacao não estão definidos. Recriando a partir de df_train_merged...")
    if 'df_train_merged' not in locals():
        print("ERRO: df_train_merged não está definido. Não é possível recriar os dados.")
    else:
        # Divisão temporal 80/20
        df_train_merged = df_train_merged.sort_values('Date')
        train_size = int(len(df_train_merged) * 0.8)
        df_treino = df_train_merged.iloc[:train_size].copy()
        df_validacao = df_train_merged.iloc[train_size:].copy()
        y_treino = df_treino['Weekly_Sales'].copy()
        y_validacao = df_validacao['Weekly_Sales'].copy()
        colunas_a_excluir = ['Weekly_Sales', 'Date']
        colunas_features = [col for col in df_treino.columns if col not in colunas_a_excluir]
        colunas_features = [f for f in colunas_features if not pd.api.types.is_datetime64_any_dtype(df_treino[f])]
        X_treino = df_treino[colunas_features].copy()
        X_validacao = df_validacao[colunas_features].copy()
        print(f"Dados recriados com sucesso. Shape de X_treino: {X_treino.shape}, Shape de X_validacao: {X_validacao.shape}")

# Verificar e tratar NaNs em X_treino e X_validacao
if 'X_treino' in locals() and 'X_validacao' in locals():
    for df_X, nome_df in zip([X_treino, X_validacao], ['X_treino', 'X_validacao']):
        nan_check = df_X.isnull().sum()
        colunas_com_nan = nan_check[nan_check > 0]
        if not colunas_com_nan.empty:
            print(f"\nColunas com NaNs em {nome_df} antes do preenchimento:")
            print(colunas_com_nan)
            print(f"Preenchendo NaNs em {nome_df}...")
            cpi_fill_value = X_treino['CPI'].median() if 'CPI' in X_treino.columns else 0
            unemp_fill_value = X_treino['Unemployment'].median() if 'Unemployment' in X_treino.columns else 0
            for col_nan in colunas_com_nan.index:
                if col_nan == 'CPI':
                    df_X[col_nan] = df_X[col_nan].fillna(cpi_fill_value)
                elif col_nan == 'Unemployment':
                    df_X[col_nan] = df_X[col_nan].fillna(unemp_fill_value)
                else:
                    df_X[col_nan] = df_X[col_nan].fillna(0)
            print(f"NaNs preenchidos em {nome_df}. Nova verificação:")
            print(df_X.isnull().sum()[df_X.isnull().sum() > 0])

# Escalonar os dados, já que o modelo foi treinado com dados escalonados
if 'X_treino' in locals() and 'X_validacao' in locals():
    if 'scaler' not in locals():
        print("\nAVISO: Scaler não encontrado. Criando um novo scaler a partir de X_treino...")
        scaler = StandardScaler()
        X_treino_scaled = scaler.fit_transform(X_treino)
        X_validacao_scaled = scaler.transform(X_validacao)
    else:
        print("\nReutilizando scaler existente...")
        X_treino_scaled = scaler.transform(X_treino)
        X_validacao_scaled = scaler.transform(X_validacao)
    print("Escalonamento concluído.")

# --- Avaliação no Conjunto de Treino ---
print("\n--- Performance no Conjunto de Treino ---")
if 'final_model' in locals() and 'X_treino_scaled' in locals() and 'y_treino' in locals():
    resultados_treino['Regressão Linear'] = calcular_metricas('Regressão Linear', final_model, X_treino_scaled, y_treino, "Treino")
else:
    print("ERRO: final_model, X_treino_scaled ou y_treino não estão definidos.")

# --- Avaliação no Conjunto de Validação ---
print("\n--- Performance no Conjunto de Validação ---")
if 'final_model' in locals() and 'X_validacao_scaled' in locals() and 'y_validacao' in locals():
    resultados_validacao['Regressão Linear'] = calcular_metricas('Regressão Linear', final_model, X_validacao_scaled, y_validacao, "Validação")
else:
    print("ERRO: final_model, X_validacao_scaled ou y_validacao não estão definidos.")

# --- Comparação Direta ---
print("\n--- Comparativo Treino vs. Validação ---")
if resultados_treino and resultados_validacao:
    df_comp_treino = pd.DataFrame(resultados_treino).T.add_suffix('_Treino')
    df_comp_validacao = pd.DataFrame(resultados_validacao).T.add_suffix('_Validação')
    
    df_comparativo_overfitting = pd.concat([df_comp_treino, df_comp_validacao], axis=1)
    # Reordenar para melhor visualização
    cols_r2 = [col for col in df_comparativo_overfitting.columns if 'R2' in col]
    cols_rmse = [col for col in df_comparativo_overfitting.columns if 'RMSE' in col]
    cols_mae = [col for col in df_comparativo_overfitting.columns if 'MAE' in col]
    
    df_comparativo_overfitting = df_comparativo_overfitting[cols_r2 + cols_rmse + cols_mae]
    display(df_comparativo_overfitting)
else:
    print("Não foi possível gerar o comparativo. Verifique se as métricas de treino e validação foram calculadas.")

# Gerando arquivo treinado

In [ ]:
import pandas as pd # Certifique-se de que o pandas está importado

print("--- Preparando Arquivo de Submissão ---")

# 1. Criar a coluna 'Id' no formato esperado (Store_Dept_Date)
# A coluna 'Date' em df_test_merged deve ser do tipo datetime.
# Vamos formatá-la como string 'YYYY-MM-DD' para criar o Id.
df_test_submission = df_test_merged.copy() # Trabalhar em uma cópia para não alterar o df_test_merged original
df_test_submission['Date_str'] = df_test_submission['Date'].dt.strftime('%Y-%m-%d')

df_test_submission['Id'] = df_test_submission['Store'].astype(str) + '_' + \
                           df_test_submission['Dept'].astype(str) + '_' + \
                           df_test_submission['Date_str']

# 2. Criar o DataFrame de submissão com as colunas 'Id' e 'Weekly_Sales'
# A coluna 'Weekly_Sales' aqui conterá suas previsões
df_submissao = pd.DataFrame({
    'Id': df_test_submission['Id'],
    'Weekly_Sales': predicoes_finais_teste  # Suas previsões finais do modelo
})

# 3. Arredondar as previsões para um número razoável de casas decimais, se desejar
# (Muitas competições não exigem, mas pode ser bom para consistência)
# df_submissao['Weekly_Sales'] = df_submissao['Weekly_Sales'].round(4)


# 4. Salvar o arquivo de submissão com o nome desejado
nome_arquivo_submissao = 'random_forest_predictions_walmart.csv'
df_submissao.to_csv(nome_arquivo_submissao, index=False)

print(f"\nArquivo de submissão '{nome_arquivo_submissao}' criado com sucesso.")

# 5. Exibir as primeiras linhas do arquivo de submissão para verificação
print("\nExemplo das primeiras 5 linhas do arquivo de submissão:")
display(df_submissao.head())

# Verificar o shape do arquivo de submissão
print(f"\nShape do arquivo de submissão: {df_submissao.shape}")

# Graficos

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score

print("\n--- Gerando Gráficos de Validação para o Modelo de Regressão Linear ---")

# Verificar se as variáveis necessárias existem
if 'final_model' not in locals() or 'X_validacao_scaled' not in locals() or 'y_validacao' not in locals():
    print("ERRO: Variáveis 'final_model', 'X_validacao_scaled' ou 'y_validacao' não estão definidas. Interrompendo.")
elif 'df_validacao' not in locals():
    print("ERRO: df_validacao não está definido. Interrompendo.")
else:
    # Gerar previsões com final_model
    y_pred = final_model.predict(X_validacao_scaled)

    # Calcular R² para usar no título
    r2 = r2_score(y_validacao, y_pred)

    # Calcular os resíduos
    residuos = y_validacao - y_pred

    # 1. Gráfico de Previsões vs. Valores Reais (Scatter Plot)
    plt.figure(figsize=(8, 6))
    plt.scatter(y_validacao, y_pred, alpha=0.5, s=10)
    plt.plot([y_validacao.min(), y_validacao.max()], [y_validacao.min(), y_validacao.max()], 'r--', lw=2, label='Linha de Perfeição')
    plt.xlabel('Valores Reais (Weekly_Sales)')
    plt.ylabel('Valores Previstos')
    plt.title(f'Previsões vs. Valores Reais (Regressão Linear)\nR²: {r2:.4f}')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # 2. Gráfico de Resíduos vs. Valores Previstos
    plt.figure(figsize=(8, 6))
    plt.scatter(y_pred, residuos, alpha=0.5, s=10)
    plt.axhline(y=0, color='r', linestyle='--', lw=2, label='Resíduo = 0')
    plt.xlabel('Valores Previstos (Weekly_Sales)')
    plt.ylabel('Resíduos')
    plt.title('Resíduos vs. Valores Previstos (Regressão Linear)')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # 3. Distribuição dos Resíduos (Histograma + KDE)
    plt.figure(figsize=(8, 6))
    sns.histplot(residuos, kde=True, stat='density')
    plt.axvline(x=0, color='r', linestyle='--', lw=1, label='Resíduo = 0')
    plt.xlabel('Resíduos')
    plt.ylabel('Densidade')
    plt.title('Distribuição dos Resíduos (Regressão Linear)')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # 4. Previsões ao Longo do Tempo
    colunas_temporais = ['DayOfYear', 'WeekOfYear', 'Month', 'Year', 'Quarter']
    coluna_temporal = 'Date' if 'Date' in df_validacao.columns else next((col for col in colunas_temporais if col in X_validacao.columns), None)

    # Filtrar para uma loja ou departamento específico
    mask = None
    if 'Store' in df_validacao.columns and 'Dept' in df_validacao.columns:
        mask = (df_validacao['Store'] == 1) & (df_validacao['Dept'] == 1)
        df_validacao_filtered = df_validacao[mask].copy()
        y_validacao_filtered = y_validacao[mask]
        y_pred_filtered = y_pred[mask]

        if len(df_validacao_filtered) == 0:
            print("AVISO: Nenhum dado encontrado para Store=1 e Dept=1. Mostrando dados sem filtro.")
            df_validacao_filtered = df_validacao
            y_validacao_filtered = y_validacao
            y_pred_filtered = y_pred
    else:
        print("AVISO: Colunas 'Store' ou 'Dept' não encontradas em df_validacao. Mostrando dados sem filtro.")
        df_validacao_filtered = df_validacao
        y_validacao_filtered = y_validacao
        y_pred_filtered = y_pred

    if coluna_temporal:
        plt.figure(figsize=(12, 6))
        if coluna_temporal == 'Date':
            plt.plot(df_validacao_filtered['Date'], y_validacao_filtered, label='Valores Reais', alpha=0.7)
            plt.plot(df_validacao_filtered['Date'], y_pred_filtered, label='Valores Previstos', alpha=0.7)
            plt.fill_between(df_validacao_filtered['Date'], y_validacao_filtered, y_pred_filtered, color='gray', alpha=0.2, label='Diferença')
            plt.gca().xaxis.set_major_locator(plt.MaxNLocator(10))
            plt.xticks(rotation=45)
        else:
            if mask is not None:
                X_validacao_filtered = X_validacao[mask]
            else:
                X_validacao_filtered = X_validacao
            plt.plot(X_validacao_filtered[coluna_temporal], y_validacao_filtered, label='Valores Reais', alpha=0.7)
            plt.plot(X_validacao_filtered[coluna_temporal], y_pred_filtered, label='Valores Previstos', alpha=0.7)
            plt.fill_between(X_validacao_filtered[coluna_temporal], y_validacao_filtered, y_pred_filtered, color='gray', alpha=0.2, label='Diferença')
            plt.xticks(rotation=45)
        plt.xlabel(coluna_temporal)
        plt.ylabel('Weekly_Sales')
        plt.title(f'Previsões vs. Valores Reais ao Longo de {coluna_temporal} (Regressão Linear)')
        plt.legend()
        plt.tight_layout()
        plt.show()
    else:
        print("AVISO: Nenhuma coluna temporal encontrada. Usando índice como proxy.")
        plt.figure(figsize=(12, 6))
        plt.plot(y_validacao_filtered, label='Valores Reais', alpha=0.7)
        plt.plot(y_pred_filtered, label='Valores Previstos', alpha=0.7)
        plt.fill_between(range(len(y_validacao_filtered)), y_validacao_filtered, y_pred_filtered, color='gray', alpha=0.2, label='Diferença')
        plt.xlabel('Índice (Proxy Temporal)')
        plt.ylabel('Weekly_Sales')
        plt.title('Previsões vs. Valores Reais ao Longo do Tempo (Regressão Linear)')
        plt.legend()
        plt.tight_layout()
        plt.show()